In [1]:
import os
os.chdir("..")

In [11]:
import pandas as pd
import datetime
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error
import argparse
import json
import numpy as np
import optuna
from utils import load_datasets, load_target, evaluate_score
from models import LightGBM, LinearRegressionWrapper, LassoWrapper, RidgeWrapper, ElasticNetWrapper, KernelRidgeWrapper, SVRWrapper



In [6]:

def train_and_predict_lightgbm(X_train_all, y_train_all, X_test, config):
    params = config['params']
    # 学習前にy_trainに、log(y+1)で変換
    y_train_all = np.log(y_train_all + 1)  # np.log1p() でもOK

    y_preds = []
    models = []
    kf = KFold(n_splits=5)
    for train_index, valid_index in kf.split(X_train_all):
        X_train, X_valid = (X_train_all.iloc[train_index, :], X_train_all.iloc[valid_index, :])
        y_train, y_valid = (y_train_all.iloc[train_index], y_train_all.iloc[valid_index])

        # lgbmの実行
        lgbm = LightGBM()
        y_pred, y_valid_pred, model = lgbm.train_and_predict(X_train, X_valid, y_train, y_valid, X_test, params)

        # 結果の保存
        y_preds.append(y_pred)
        models.append(model)

    # CVスコア
    scores = [
        m.best_score['valid_0'][config['loss']] for m in models
    ]
    score = sum(scores) / len(scores)
    return score

def train_and_predict_linear(X_train_all, y_train_all, X_test, config):

    params = config['params']
    
    # 学習前にy_trainに、log(y+1)で変換
    y_train_all = np.log(y_train_all + 1)  # np.log1p() でもOK

    y_preds = []
    scores = []  # CVスコア
    kf = KFold(n_splits=5)
    for train_index, valid_index in kf.split(X_train_all):
        X_train, X_valid = (X_train_all.iloc[train_index, :], X_train_all.iloc[valid_index, :])
        y_train, y_valid = (y_train_all.iloc[train_index], y_train_all.iloc[valid_index])

        if config['model'] == "LinearRegression":
            lr = LinearRegressionWrapper()
        elif config['model'] == "Lasso":
            lr = LassoWrapper()
        elif config['model'] == "Ridge":
            lr = RidgeWrapper()
        elif config['model'] == "ErasticNet":
            lr = ElasticNetWrapper()
        elif config['model'] == "KernelRidge":
            lr = KernelRidgeWrapper()
        elif config['model'] == "SVR":
            lr = SVRWrapper()

        y_pred, y_valid_pred, m = lr.train_and_predict(X_train, X_valid, y_train, y_valid, X_test, params)

        # 結果の保存
        y_preds.append(y_pred)

        # スコア
        rmse_valid = evaluate_score(y_valid, y_valid_pred, config['loss'])
        scores.append(rmse_valid)

    score = sum(scores) / len(scores)

    return score


# Lasso

In [44]:


# 1. Define an objective function to be maximized.
def objective(trial):
    config = {
            "model": "Ridge",
            "features": [
                "numerical_features",
                "object_features"
                ],
            "params": {
                "alpha" : trial.suggest_loguniform('alpha', 1e-15, 100)

            },
            "loss": "rmse",
            "target_name": "SalePrice",
            "ID_name": "Id"
    }
    feats = config['features']
    target_name = config['target_name']

    # 指定した特徴量からデータをロード
    X_train_all, X_test = load_datasets(feats)
    y_train_all = load_target(target_name)
    if config['model'] == 'LightGBM':
        return train_and_predict_lightgbm(X_train_all, y_train_all, X_test, config)
    elif config['model'] in ['LinearRegression', 'Lasso', 'Ridge', 'ErasticNet', 'KernelRidge']:
        return train_and_predict_linear(X_train_all, y_train_all, X_test, config)
        

In [42]:

# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2020-11-21 08:28:42,430] A new study created in memory with name: no-name-a295d735-f9f7-49f7-b82e-9671bb1a53e5
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.81622217189963, tolerance: 0.01875856904679318
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.708900935075422, tolerance: 0.017638952492142118
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.5360806787621, tolerance: 0.018034543670964208
  positive)
/usr/local/var/pyenv

/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.7147344097825883, tolerance: 0.018660924246928393
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.679280535517232, tolerance: 0.018414742906523215
  positive)
[I 2020-11-21 08:28:55,241] Trial 7 finished with value: 0.11431315918329055 and parameters: {'alpha': 5.009946686486284e-12}. Best is trial 4 with value: 0.10061349198263705.
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.78547238667901

/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.267782039572796, tolerance: 0.018414742906523215
  positive)
[I 2020-11-21 08:29:07,558] Trial 17 finished with value: 0.11254720312435854 and parameters: {'alpha': 7.947506071402209e-06}. Best is trial 4 with value: 0.10061349198263705.
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.8146975365455886, tolerance: 0.01875856904679318
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.70849036415773

[I 2020-11-21 08:29:20,043] Trial 27 finished with value: 0.10252199876489532 and parameters: {'alpha': 0.000995306009463438}. Best is trial 4 with value: 0.10061349198263705.
[I 2020-11-21 08:29:20,680] Trial 28 finished with value: 0.30885958674621045 and parameters: {'alpha': 0.17376495523733165}. Best is trial 4 with value: 0.10061349198263705.
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.27164930943027965, tolerance: 0.017638952492142118
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.06774986502472835, tolerance: 0.018034543670964208
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/s

/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.4926669928642804, tolerance: 0.018034543670964208
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.6309710047341635, tolerance: 0.018660924246928393
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.6370516584525188, tolerance: 0.018414742906523215
  positive)
[I 2020-11-21 08:29:34,536] Trial 38 finished with value: 0.11404035657070762 and parameters: {'alpha': 8.897474302260418e-07}.

/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.50108152958729, tolerance: 0.018660924246928393
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.560013651728242, tolerance: 0.018414742906523215
  positive)
[I 2020-11-21 08:29:47,387] Trial 49 finished with value: 0.1136703523459123 and parameters: {'alpha': 2.470608520066647e-06}. Best is trial 4 with value: 0.10061349198263705.
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.816065873136728,

/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.5360854195642313, tolerance: 0.018034543670964208
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.714734777220104, tolerance: 0.018660924246928393
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.6792807068490134, tolerance: 0.018414742906523215
  positive)
[I 2020-11-21 08:29:58,573] Trial 57 finished with value: 0.11431314058578039 and parameters: {'alpha': 1.3087851027378585e-12}.

/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.4464433172135407, tolerance: 0.018660924246928393
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.5203755600780062, tolerance: 0.018414742906523215
  positive)
[I 2020-11-21 08:30:12,088] Trial 68 finished with value: 0.11349937930207812 and parameters: {'alpha': 3.246350692826362e-06}. Best is trial 4 with value: 0.10061349198263705.
[I 2020-11-21 08:30:13,143] Trial 69 finished with value: 0.10061424593603978 and parameters: {'alpha': 0.00046350741565734924}. Best is trial 4 with value: 0.10061349198263705.
[I 2020-11-21 08:30:14,036] Trial 70 finished with value: 0.108

/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.310704733967191, tolerance: 0.018414742906523215
  positive)
[I 2020-11-21 08:30:28,891] Trial 83 finished with value: 0.11269575076653093 and parameters: {'alpha': 7.156580412004382e-06}. Best is trial 4 with value: 0.10061349198263705.
[I 2020-11-21 08:30:29,647] Trial 84 finished with value: 0.10968713438819633 and parameters: {'alpha': 0.002277491570008347}. Best is trial 4 with value: 0.10061349198263705.
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.750134234870714, tolerance: 0.01875856904679318
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/skle

/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.6313036658419136, tolerance: 0.017638952492142118
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.457462952014179, tolerance: 0.018034543670964208
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.570999525351308, tolerance: 0.018660924246928393
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarni

In [43]:
study.best_trial

FrozenTrial(number=93, value=0.10061286670395464, datetime_start=datetime.datetime(2020, 11, 21, 8, 30, 40, 413833), datetime_complete=datetime.datetime(2020, 11, 21, 8, 30, 41, 572286), params={'alpha': 0.00044227928994987245}, distributions={'alpha': LogUniformDistribution(high=10, low=1e-15)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=93, state=TrialState.COMPLETE)

# Ridge

In [46]:

# 1. Define an objective function to be maximized.
def objective(trial):
    config = {
            "model": "Ridge",
            "features": [
                "numerical_features",
                "object_features"
                ],
            "params": {
                "alpha" : trial.suggest_loguniform('alpha', 1e-15, 100)

            },
            "loss": "rmse",
            "target_name": "SalePrice",
            "ID_name": "Id"
    }
    feats = config['features']
    target_name = config['target_name']

    # 指定した特徴量からデータをロード
    X_train_all, X_test = load_datasets(feats)
    y_train_all = load_target(target_name)
    if config['model'] == 'LightGBM':
        return train_and_predict_lightgbm(X_train_all, y_train_all, X_test, config)
    elif config['model'] in ['LinearRegression', 'Lasso', 'Ridge', 'ErasticNet', 'KernelRidge']:
        return train_and_predict_linear(X_train_all, y_train_all, X_test, config)

# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

In [48]:
study.best_trial

FrozenTrial(number=96, value=0.1033346156851342, datetime_start=datetime.datetime(2020, 11, 21, 8, 34, 33, 890435), datetime_complete=datetime.datetime(2020, 11, 21, 8, 34, 34, 534665), params={'alpha': 13.97332056206028}, distributions={'alpha': LogUniformDistribution(high=100, low=1e-15)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=96, state=TrialState.COMPLETE)

# Elastic Net

In [52]:

# 1. Define an objective function to be maximized.
def objective(trial):
    config = {
            "model": "ErasticNet",
            "features": [
                "numerical_features",
                "object_features"
                ],
            "params": {
                "alpha" : trial.suggest_loguniform('alpha', 1e-15, 100),
                "l1_ratio": trial.suggest_loguniform('l1_ratio', 1e-5, 10)

            },
            "loss": "rmse",
            "target_name": "SalePrice",
            "ID_name": "Id"
    }

    feats = config['features']
    target_name = config['target_name']
    # 指定した特徴量からデータをロード
    X_train_all, X_test = load_datasets(feats)
    y_train_all = load_target(target_name)
    if config['model'] == 'LightGBM':
        return train_and_predict_lightgbm(X_train_all, y_train_all, X_test, config)
    elif config['model'] in ['LinearRegression', 'Lasso', 'Ridge', 'ErasticNet', 'KernelRidge']:
        return train_and_predict_linear(X_train_all, y_train_all, X_test, config)

# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2020-11-21 08:40:06,933] A new study created in memory with name: no-name-ccaa9f2c-84e2-45f1-82c3-6cd8ba85800f
[I 2020-11-21 08:40:07,705] Trial 0 finished with value: 0.2709490801822532 and parameters: {'alpha': 12.243571498341566, 'l1_ratio': 1.3028575501954168e-05}. Best is trial 0 with value: 0.2709490801822532.
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.8162432218409483, tolerance: 0.01875856904679318
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.7089073100360936, tolerance: 0.017638952492142118
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_d

/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.6792807636279283, tolerance: 0.018414742906523215
  positive)
[I 2020-11-21 08:40:18,565] Trial 6 finished with value: 0.11431313445054248 and parameters: {'alpha': 2.655700578155715e-13, 'l1_ratio': 0.3224682005248994}. Best is trial 6 with value: 0.11431313445054248.
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.815964902427529, tolerance: 0.01875856904679318
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations

/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.5389807206236283, tolerance: 0.018034543670964208
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.717859324792965, tolerance: 0.018660924246928393
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.682248892609802, tolerance: 0.018414742906523215
  positive)
[I 2020-11-21 08:40:32,510] Trial 14 finished with value: 0.11418893122571669 and parameters: {'alpha': 3.7065268595574312e-06, '

/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.702386825158928, tolerance: 0.018414742906523215
  positive)
[I 2020-11-21 08:40:45,432] Trial 23 finished with value: 0.11309853591006125 and parameters: {'alpha': 4.266061037183296e-05, 'l1_ratio': 0.0022779737114705613}. Best is trial 12 with value: 0.10337344419791691.
[I 2020-11-21 08:40:46,614] Trial 24 finished with value: 0.10333330942652734 and parameters: {'alpha': 0.0151340814909565, 'l1_ratio': 0.0005057299328132176}. Best is trial 24 with value: 0.10333330942652734.
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.836745963826277, tolerance: 0.01875856904679318
  positive

[I 2020-11-21 08:41:00,770] Trial 34 finished with value: 0.11997678038815718 and parameters: {'alpha': 0.21269472378625584, 'l1_ratio': 0.012763729916737037}. Best is trial 32 with value: 0.10323007513323941.
[I 2020-11-21 08:41:01,812] Trial 35 finished with value: 0.10722931834497292 and parameters: {'alpha': 0.01185271583005807, 'l1_ratio': 0.13042988241096024}. Best is trial 32 with value: 0.10323007513323941.
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.8181478670339404, tolerance: 0.01875856904679318
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.712647176645778, tolerance: 0.017638952492142118
  positive)
/

/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.7990293038694767, tolerance: 0.01875856904679318
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.699078847484567, tolerance: 0.017638952492142118
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.521361403041432, tolerance: 0.018034543670964208
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarnin

/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.7155819097540768, tolerance: 0.017638952492142118
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.5447529670825135, tolerance: 0.018034543670964208
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.7237197043044157, tolerance: 0.018660924246928393
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWar

[I 2020-11-21 08:41:41,712] Trial 63 finished with value: 0.10680514958302836 and parameters: {'alpha': 0.07544888617175997, 'l1_ratio': 9.043181787793187e-05}. Best is trial 32 with value: 0.10323007513323941.
[I 2020-11-21 08:41:43,249] Trial 64 finished with value: 0.1033813042730575 and parameters: {'alpha': 0.009229845028984232, 'l1_ratio': 0.00018223912132182658}. Best is trial 32 with value: 0.10323007513323941.
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.4219080821150154, tolerance: 0.01875856904679318
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.758285873132046, tolerance: 0.017638952492142118
  positiv

/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.760917507722585, tolerance: 0.018414742906523215
  positive)
[I 2020-11-21 08:41:58,989] Trial 75 finished with value: 0.10876044428137477 and parameters: {'alpha': 0.0005349474374457317, 'l1_ratio': 0.0005592615923698878}. Best is trial 32 with value: 0.10323007513323941.
[I 2020-11-21 08:42:00,845] Trial 76 finished with value: 0.10335510431351425 and parameters: {'alpha': 0.00827925480717639, 'l1_ratio': 0.001216972358756709}. Best is trial 32 with value: 0.10323007513323941.
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.854505886427683, tolerance: 0.01875856904679318
  positive

[I 2020-11-21 08:42:26,771] Trial 91 finished with value: 0.10328487128617495 and parameters: {'alpha': 0.015766200686644295, 'l1_ratio': 0.0015099949946666368}. Best is trial 32 with value: 0.10323007513323941.
[I 2020-11-21 08:42:28,280] Trial 92 finished with value: 0.10338588920252863 and parameters: {'alpha': 0.007565877042664609, 'l1_ratio': 0.0015333099619739717}. Best is trial 32 with value: 0.10323007513323941.
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.8058210159395847, tolerance: 0.01875856904679318
  positive)
/usr/local/var/pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.6214559441012475, tolerance: 0.017638952492142118
  posit

In [53]:
study.best_trial

FrozenTrial(number=32, value=0.10323007513323941, datetime_start=datetime.datetime(2020, 11, 21, 8, 40, 56, 456995), datetime_complete=datetime.datetime(2020, 11, 21, 8, 40, 57, 973190), params={'alpha': 0.008134728198737267, 'l1_ratio': 0.0029556742579328124}, distributions={'alpha': LogUniformDistribution(high=100, low=1e-15), 'l1_ratio': LogUniformDistribution(high=10, low=1e-05)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=32, state=TrialState.COMPLETE)

# kernel ridge (poly)

In [55]:
# 1. Define an objective function to be maximized.
def objective(trial):
    config = {
            "model": "KernelRidge",
            "features": [
                "numerical_features",
                "object_features"
                ],
            "params": {
                "alpha" : trial.suggest_loguniform('alpha', 1e-5, 100),
                "kernel" : 'poly',
                "degree" : trial.suggest_int('degree', 2, 4),
                "coef0": trial.suggest_loguniform('coef0', 1e-4, 10)

            },
            "loss": "rmse",
            "target_name": "SalePrice",
            "ID_name": "Id"
    }

    feats = config['features']
    target_name = config['target_name']
    # 指定した特徴量からデータをロード
    X_train_all, X_test = load_datasets(feats)
    y_train_all = load_target(target_name)
    if config['model'] == 'LightGBM':
        return train_and_predict_lightgbm(X_train_all, y_train_all, X_test, config)
    elif config['model'] in ['LinearRegression', 'Lasso', 'Ridge', 'ErasticNet', 'KernelRidge']:
        return train_and_predict_linear(X_train_all, y_train_all, X_test, config)

# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2020-11-21 08:45:28,552] A new study created in memory with name: no-name-69e9149c-a868-413f-b3ff-bca8c93ccdeb
[I 2020-11-21 08:45:30,195] Trial 0 finished with value: 5.729316343058227 and parameters: {'alpha': 0.11008125303523662, 'degree': 3, 'coef0': 0.01081862760809356}. Best is trial 0 with value: 5.729316343058227.
[I 2020-11-21 08:45:31,839] Trial 1 finished with value: 4.195135762079753 and parameters: {'alpha': 0.02014836055340176, 'degree': 3, 'coef0': 0.003330513756887406}. Best is trial 1 with value: 4.195135762079753.
[I 2020-11-21 08:45:33,423] Trial 2 finished with value: 3.9285933261176114 and parameters: {'alpha': 1.9274948404147034e-05, 'degree': 4, 'coef0': 0.0009621907228265197}. Best is trial 2 with value: 3.9285933261176114.
[I 2020-11-21 08:45:34,958] Trial 3 finished with value: 11.068964248650701 and parameters: {'alpha': 12.103920972563515, 'degree': 4, 'coef0': 0.0029421175556205168}. Best is trial 2 with value: 3.9285933261176114.
[I 2020-11-21 08:45:36,

[I 2020-11-21 08:46:21,713] Trial 36 finished with value: 0.10558295702622952 and parameters: {'alpha': 7.364466795177675, 'degree': 4, 'coef0': 4.62946092975351}. Best is trial 12 with value: 0.10320502707772308.
[I 2020-11-21 08:46:23,246] Trial 37 finished with value: 10.792690836131353 and parameters: {'alpha': 21.918347503382577, 'degree': 3, 'coef0': 0.011015497660465081}. Best is trial 12 with value: 0.10320502707772308.
[I 2020-11-21 08:46:24,792] Trial 38 finished with value: 6.607431453088107 and parameters: {'alpha': 0.0955326746656912, 'degree': 3, 'coef0': 0.00016313818978307953}. Best is trial 12 with value: 0.10320502707772308.
[I 2020-11-21 08:46:26,301] Trial 39 finished with value: 0.20710267660168008 and parameters: {'alpha': 0.010918852726148065, 'degree': 4, 'coef0': 0.25568938918111866}. Best is trial 12 with value: 0.10320502707772308.
[I 2020-11-21 08:46:27,370] Trial 40 finished with value: 0.10515519771390713 and parameters: {'alpha': 1.8715168733306096, 'degr

[I 2020-11-21 08:47:17,946] Trial 72 finished with value: 0.1060364111826095 and parameters: {'alpha': 7.731450812824452, 'degree': 3, 'coef0': 3.2192028857109602}. Best is trial 52 with value: 0.1031500943094874.
[I 2020-11-21 08:47:19,497] Trial 73 finished with value: 0.10366204285160689 and parameters: {'alpha': 3.741452365544266, 'degree': 3, 'coef0': 6.798190098897142}. Best is trial 52 with value: 0.1031500943094874.
[I 2020-11-21 08:47:21,048] Trial 74 finished with value: 0.1408681704149991 and parameters: {'alpha': 0.002662064893918104, 'degree': 3, 'coef0': 9.517227764255702}. Best is trial 52 with value: 0.1031500943094874.
[I 2020-11-21 08:47:22,546] Trial 75 finished with value: 0.104102797526392 and parameters: {'alpha': 10.427531419049075, 'degree': 3, 'coef0': 4.952266716364914}. Best is trial 52 with value: 0.1031500943094874.
[I 2020-11-21 08:47:24,190] Trial 76 finished with value: 0.12913189105475573 and parameters: {'alpha': 31.158890338539045, 'degree': 3, 'coef0

In [56]:
study.best_trial

FrozenTrial(number=83, value=0.10305224643848125, datetime_start=datetime.datetime(2020, 11, 21, 8, 47, 34, 60205), datetime_complete=datetime.datetime(2020, 11, 21, 8, 47, 35, 713292), params={'alpha': 9.2304694336833, 'degree': 3, 'coef0': 7.405346770365897}, distributions={'alpha': LogUniformDistribution(high=100, low=1e-05), 'degree': IntUniformDistribution(high=4, low=2, step=1), 'coef0': LogUniformDistribution(high=10, low=0.0001)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=83, state=TrialState.COMPLETE)

# kernel ridge (rbf)
複雑過ぎてダメそう

In [8]:
# 1. Define an objective function to be maximized.
def objective(trial):
    config = {
            "model": "KernelRidge",
            "features": [
                "numerical_features",
                "object_features"
                ],
            "params": {
                "alpha" : trial.suggest_loguniform('alpha', 1e-5, 100),
                "kernel" : 'rbf',
                "degree": 2, # 意味無しだが、無いとエラーに
                "coef0": trial.suggest_loguniform('coef0', 1e-4, 10)

            },
            "loss": "rmse",
            "target_name": "SalePrice",
            "ID_name": "Id"
    }
    feats = config['features']
    target_name = config['target_name']

    # 指定した特徴量からデータをロード
    X_train_all, X_test = load_datasets(feats)
    y_train_all = load_target(target_name)
    if config['model'] == 'LightGBM':
        return train_and_predict_lightgbm(X_train_all, y_train_all, X_test, config)
    elif config['model'] in ['LinearRegression', 'Lasso', 'Ridge', 'ErasticNet', 'KernelRidge', 'SVR']:
        return train_and_predict_linear(X_train_all, y_train_all, X_test, config)

# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2020-11-21 09:09:58,632] A new study created in memory with name: no-name-3b76c2ec-7278-4792-9fd2-a5976e75bbbd
[I 2020-11-21 09:10:00,387] Trial 0 finished with value: 2.059916792529886 and parameters: {'alpha': 58.418500493925556, 'coef0': 0.15599901052077841}. Best is trial 0 with value: 2.059916792529886.
[I 2020-11-21 09:10:01,813] Trial 1 finished with value: 0.27256758029040284 and parameters: {'alpha': 0.10020407497906486, 'coef0': 0.6810480667996159}. Best is trial 1 with value: 0.27256758029040284.
[I 2020-11-21 09:10:03,202] Trial 2 finished with value: 0.22246882757714684 and parameters: {'alpha': 0.0015292261203993618, 'coef0': 0.21998294974984267}. Best is trial 2 with value: 0.22246882757714684.
[I 2020-11-21 09:10:04,548] Trial 3 finished with value: 0.38837191535867305 and parameters: {'alpha': 0.6590750852952686, 'coef0': 0.6079627001973277}. Best is trial 2 with value: 0.22246882757714684.
[I 2020-11-21 09:10:05,830] Trial 4 finished with value: 0.25261368162234504

[I 2020-11-21 09:10:49,718] Trial 37 finished with value: 0.2221726154407843 and parameters: {'alpha': 0.00039607344498090297, 'coef0': 0.26750632526579515}. Best is trial 31 with value: 0.22215306600109147.
[I 2020-11-21 09:10:51,124] Trial 38 finished with value: 0.2494472716977691 and parameters: {'alpha': 0.04425978960157087, 'coef0': 0.0048919108871860245}. Best is trial 31 with value: 0.22215306600109147.
[I 2020-11-21 09:10:52,423] Trial 39 finished with value: 0.22478827835548948 and parameters: {'alpha': 0.004812239305797234, 'coef0': 0.11798902333511407}. Best is trial 31 with value: 0.22215306600109147.
[I 2020-11-21 09:10:53,710] Trial 40 finished with value: 0.22241902583527606 and parameters: {'alpha': 0.0014328649838112077, 'coef0': 0.000353118069853339}. Best is trial 31 with value: 0.22215306600109147.
[I 2020-11-21 09:10:55,016] Trial 41 finished with value: 0.22215943763467577 and parameters: {'alpha': 0.00046463491214232, 'coef0': 0.44893770039808806}. Best is trial

[I 2020-11-21 09:11:38,419] Trial 75 finished with value: 0.22217445314636203 and parameters: {'alpha': 0.00038888731237845484, 'coef0': 1.14927472390617}. Best is trial 31 with value: 0.22215306600109147.
[I 2020-11-21 09:11:39,625] Trial 76 finished with value: 0.22223810853282994 and parameters: {'alpha': 0.0010124927617846083, 'coef0': 0.18583812091262586}. Best is trial 31 with value: 0.22215306600109147.
[I 2020-11-21 09:11:40,844] Trial 77 finished with value: 0.2223821303868644 and parameters: {'alpha': 7.675526903667977e-05, 'coef0': 0.08948748607174693}. Best is trial 31 with value: 0.22215306600109147.
[I 2020-11-21 09:11:42,117] Trial 78 finished with value: 0.22346723524187845 and parameters: {'alpha': 0.0030635970187411765, 'coef0': 0.1197946114900947}. Best is trial 31 with value: 0.22215306600109147.
[I 2020-11-21 09:11:43,393] Trial 79 finished with value: 0.22256508713567164 and parameters: {'alpha': 0.0017040832558400934, 'coef0': 0.5242612333940431}. Best is trial 3

In [9]:
study.best_trial

FrozenTrial(number=80, value=0.22215243327751505, datetime_start=datetime.datetime(2020, 11, 21, 9, 11, 43, 394357), datetime_complete=datetime.datetime(2020, 11, 21, 9, 11, 44, 659192), params={'alpha': 0.0005741952109157125, 'coef0': 3.754170390708927}, distributions={'alpha': LogUniformDistribution(high=100, low=1e-05), 'coef0': LogUniformDistribution(high=10, low=0.0001)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=80, state=TrialState.COMPLETE)

# SVR

In [12]:
# 1. Define an objective function to be maximized.
def objective(trial):
    config = {
            "model": "SVR",
            "features": [
                "numerical_features",
                "object_features"
                ],
            "params": {
                "kernel":"poly",
                "degree": trial.suggest_int('degree', 2, 4),
                "coef0": trial.suggest_loguniform('coef0', 1e-4, 10),
                "C":trial.suggest_uniform('C', 0, 2.0),
                "epsilon": trial.suggest_loguniform('epsilon', 1e-5, 1)

            },
            "loss": "rmse",
            "target_name": "SalePrice",
            "ID_name": "Id"
    }
    feats = config['features']
    target_name = config['target_name']

    # 指定した特徴量からデータをロード
    X_train_all, X_test = load_datasets(feats)
    y_train_all = load_target(target_name)
    if config['model'] == 'LightGBM':
        return train_and_predict_lightgbm(X_train_all, y_train_all, X_test, config)
    elif config['model'] in ['LinearRegression', 'Lasso', 'Ridge', 'ErasticNet', 'KernelRidge', 'SVR']:
        return train_and_predict_linear(X_train_all, y_train_all, X_test, config)

# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2020-11-21 09:15:24,119] A new study created in memory with name: no-name-5c2c9372-439f-4005-b432-fccd3c84d410
[I 2020-11-21 09:15:38,306] Trial 0 finished with value: 0.13097691912541237 and parameters: {'degree': 4, 'coef0': 4.691430744876913, 'C': 1.9539292586411596, 'epsilon': 0.0018125620092745707}. Best is trial 0 with value: 0.13097691912541237.
[I 2020-11-21 09:15:46,613] Trial 1 finished with value: 0.20300234617621263 and parameters: {'degree': 4, 'coef0': 0.0002159024580783942, 'C': 1.8326749076025461, 'epsilon': 0.0004463383997343004}. Best is trial 0 with value: 0.13097691912541237.
[I 2020-11-21 09:15:54,706] Trial 2 finished with value: 0.23949986784146943 and parameters: {'degree': 4, 'coef0': 0.000839539393376443, 'C': 0.6233285588669752, 'epsilon': 0.0021832854035424717}. Best is trial 0 with value: 0.13097691912541237.
[I 2020-11-21 09:16:03,026] Trial 3 finished with value: 0.15982672495251643 and parameters: {'degree': 4, 'coef0': 0.06803708974317828, 'C': 1.277

[I 2020-11-21 09:19:36,637] Trial 32 finished with value: 0.1072930290338681 and parameters: {'degree': 2, 'coef0': 2.249723477833821, 'C': 0.714351548769813, 'epsilon': 0.016107295657564694}. Best is trial 32 with value: 0.1072930290338681.
[I 2020-11-21 09:19:43,085] Trial 33 finished with value: 0.10722188499279761 and parameters: {'degree': 2, 'coef0': 5.311064426562288, 'C': 0.611016824981415, 'epsilon': 0.04377935010630321}. Best is trial 33 with value: 0.10722188499279761.
[I 2020-11-21 09:19:49,097] Trial 34 finished with value: 0.10605481665576762 and parameters: {'degree': 2, 'coef0': 2.67705431973581, 'C': 0.6339134888897136, 'epsilon': 0.04645210942679866}. Best is trial 34 with value: 0.10605481665576762.
[I 2020-11-21 09:19:49,930] Trial 35 finished with value: 0.154924222560829 and parameters: {'degree': 4, 'coef0': 4.904754491540857, 'C': 0.6206900521210983, 'epsilon': 0.3929678086550641}. Best is trial 34 with value: 0.10605481665576762.
[I 2020-11-21 09:19:51,042] Tri

[I 2020-11-21 09:22:34,458] Trial 64 finished with value: 0.10351245551685986 and parameters: {'degree': 2, 'coef0': 0.9052221084294821, 'C': 0.25417778087639975, 'epsilon': 0.024719617688751812}. Best is trial 41 with value: 0.10257032668815132.
[I 2020-11-21 09:22:42,418] Trial 65 finished with value: 0.17181265845799032 and parameters: {'degree': 2, 'coef0': 1.13980457133137, 'C': 0.0036452975711258606, 'epsilon': 0.001847407180699512}. Best is trial 41 with value: 0.10257032668815132.
[I 2020-11-21 09:22:49,493] Trial 66 finished with value: 0.1047170302681486 and parameters: {'degree': 2, 'coef0': 2.716053257690923, 'C': 0.42570330549086427, 'epsilon': 0.018795798105165254}. Best is trial 41 with value: 0.10257032668815132.
[I 2020-11-21 09:22:57,387] Trial 67 finished with value: 0.14472038418991834 and parameters: {'degree': 2, 'coef0': 0.013641602234551674, 'C': 0.07775045331078159, 'epsilon': 0.005478577402383002}. Best is trial 41 with value: 0.10257032668815132.
[I 2020-11-2

[I 2020-11-21 09:26:23,833] Trial 96 finished with value: 0.10207493707111881 and parameters: {'degree': 2, 'coef0': 7.865345479478388, 'C': 0.09111385520064191, 'epsilon': 0.0026048466307931843}. Best is trial 96 with value: 0.10207493707111881.
[I 2020-11-21 09:26:33,009] Trial 97 finished with value: 0.10327487898175984 and parameters: {'degree': 2, 'coef0': 7.452118874609367, 'C': 0.16996223643524255, 'epsilon': 0.0025488741622280166}. Best is trial 96 with value: 0.10207493707111881.
[I 2020-11-21 09:26:41,194] Trial 98 finished with value: 0.1029619389249424 and parameters: {'degree': 2, 'coef0': 4.277585892183445, 'C': 0.07176719321092725, 'epsilon': 0.0007520142094004234}. Best is trial 96 with value: 0.10207493707111881.
[I 2020-11-21 09:26:51,377] Trial 99 finished with value: 0.1291048866963515 and parameters: {'degree': 2, 'coef0': 0.00010306560700002056, 'C': 0.16421030757523208, 'epsilon': 0.0027792546077877677}. Best is trial 96 with value: 0.10207493707111881.


In [13]:
study.best_trial

FrozenTrial(number=96, value=0.10207493707111881, datetime_start=datetime.datetime(2020, 11, 21, 9, 26, 15, 423265), datetime_complete=datetime.datetime(2020, 11, 21, 9, 26, 23, 833224), params={'degree': 2, 'coef0': 7.865345479478388, 'C': 0.09111385520064191, 'epsilon': 0.0026048466307931843}, distributions={'degree': IntUniformDistribution(high=4, low=2, step=1), 'coef0': LogUniformDistribution(high=10, low=0.0001), 'C': UniformDistribution(high=2.0, low=0), 'epsilon': LogUniformDistribution(high=1, low=1e-05)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=96, state=TrialState.COMPLETE)